# Projeto 2 — Aprendizado de Máquina (Olist)

## 1) Definição do problema

Objetivo: construir um modelo de classificação binária para prever se um pedido será entregue com atraso em relação à data estimada no momento da compra.

Alvo (y): atraso_entrega = 1 se `order_delivered_customer_date` > `order_estimated_delivery_date`, caso contrário 0.

Critério de sucesso: priorizar F1-Score (equilíbrio entre precisão e revocação), reportando também precisão, revocação, acurácia e ROC-AUC.

Métricas (fórmulas):
- Precisão = TP / (TP + FP)
- Revocação = TP / (TP + FN)
- F1 = 2 * (precisão * revocação) / (precisão + revocação)
- AUC calculada a partir da curva ROC

Restrições/assunções:
- Usar apenas variáveis disponíveis até a compra/aprovação (evitar vazamento de informação).
- Reprodutibilidade via semente aleatória fixa.

Notas:
- Base: arquivos CSV oficiais do repositório Olist na pasta atual.
- Vamos agregar características no nível do pedido (order_id) a partir de itens, produtos, pagamentos e perfis de cliente/vendedor.

In [1]:
# Imports, semente e caminhos
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, classification_report, confusion_matrix, RocCurveDisplay, PrecisionRecallDisplay
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import joblib

# Reprodutibilidade
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Caminhos
BASE_DIR = Path('.')
CSV_ORDERS = BASE_DIR / 'olist_orders_dataset.csv'
CSV_ORDER_ITEMS = BASE_DIR / 'olist_order_items_dataset.csv'
CSV_PRODUCTS = BASE_DIR / 'olist_products_dataset.csv'
CSV_CUSTOMERS = BASE_DIR / 'olist_customers_dataset.csv'
CSV_SELLERS = BASE_DIR / 'olist_sellers_dataset.csv'
CSV_PAYMENTS = BASE_DIR / 'olist_order_payments_dataset.csv'
CSV_REVIEWS = BASE_DIR / 'olist_order_reviews_dataset.csv'
CSV_GEO = BASE_DIR / 'olist_geolocation_dataset.csv'

# Saídas
MODELS_DIR = BASE_DIR / 'models'
OUTPUT_DIR = BASE_DIR / 'output'
MODELS_DIR.mkdir(exist_ok=True)
OUTPUT_DIR.mkdir(exist_ok=True)

print('Arquivos encontrados?', CSV_ORDERS.exists(), CSV_ORDER_ITEMS.exists())

Arquivos encontrados? True True


## 2) Pré-processamento de dados

Passos desta seção:
- Carregar CSVs (orders, items, products, customers, sellers, payments, reviews, geolocation).
- Selecionar colunas úteis e realizar merges no nível do pedido (order_id).
- Criar o alvo `atraso_entrega` e remover colunas pós-entrega para evitar vazamento.
- Engenhar features disponíveis até compra/aprovação (ex.: tempo estimado, total preço/frete, nº itens, categoria, UF, método de pagamento, parcelas).
- Listar colunas numéricas e categóricas para o pré-processador.

In [3]:
# Carregar dados brutos
def load_data():
    orders = pd.read_csv(CSV_ORDERS)
    order_items = pd.read_csv(CSV_ORDER_ITEMS)
    products = pd.read_csv(CSV_PRODUCTS)
    customers = pd.read_csv(CSV_CUSTOMERS)
    sellers = pd.read_csv(CSV_SELLERS)
    payments = pd.read_csv(CSV_PAYMENTS)
    reviews = pd.read_csv(CSV_REVIEWS)
    # geolocation é pesado e granular em CEP; manteremos fora por simplicidade inicial
    return orders, order_items, products, customers, sellers, payments, reviews

orders, order_items, products, customers, sellers, payments, reviews = load_data()

orders.head(), order_items.head()

(                           order_id                       customer_id  \
 0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
 1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
 2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
 3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
 4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   
 
   order_status order_purchase_timestamp    order_approved_at  \
 0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
 1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
 2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
 3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
 4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   
 
   order_delivered_carrier_date order_delivered_customer_date  \
 0          2017-10-04 19:55:00           2017-10-10 21:25:13   
 1          2018-07-26 14:31:00 

In [4]:
# Conversão de datas e criação do alvo (y)

def to_datetime(df, cols):
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce')

# Converter datas relevantes
order_date_cols = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

to_datetime(orders, order_date_cols)

# Alvo: atraso_entrega (1 se entregue após a data estimada)
orders['atraso_entrega'] = (
    orders['order_delivered_customer_date'] > orders['order_estimated_delivery_date']
).astype('Int64')

# Manter apenas pedidos com datas necessárias para definir o alvo
orders = orders.dropna(subset=['order_delivered_customer_date', 'order_estimated_delivery_date'])

orders['atraso_entrega'].value_counts(dropna=False), orders[order_date_cols].isna().mean().sort_values(ascending=False).head(10)

(atraso_entrega
 0    88649
 1     7827
 Name: count, dtype: Int64,
 order_approved_at                0.000145
 order_delivered_carrier_date     0.000010
 order_purchase_timestamp         0.000000
 order_delivered_customer_date    0.000000
 order_estimated_delivery_date    0.000000
 dtype: float64)

In [5]:
# Selecionar features disponíveis até a compra/aprovação e agregar por pedido

# Itens do pedido: somatórios e contagens
item_aggs = order_items.groupby('order_id').agg(
    total_preco=('price', 'sum'),
    total_frete=('freight_value', 'sum'),
    num_itens=('order_item_id', 'count'),
    num_vendedores=('seller_id', pd.Series.nunique),
    num_produtos=('product_id', pd.Series.nunique)
).reset_index()

# Produtos: características físicas (médias por pedido via merge nos itens)
prod_cols = ['product_id','product_category_name','product_weight_g','product_length_cm','product_height_cm','product_width_cm']
order_items_prod = order_items.merge(products[prod_cols], on='product_id', how='left')
prod_aggs = order_items_prod.groupby('order_id').agg(
    peso_medio=('product_weight_g','mean'),
    vol_medio_cm3=(lambda x: (order_items_prod['product_length_cm']*order_items_prod['product_height_cm']*order_items_prod['product_width_cm']).groupby(order_items_prod['order_id']).mean())
)
# A expressão acima complica dentro do agg; calcular volume explicitamente
order_items_prod['volume_cm3'] = order_items_prod[['product_length_cm','product_height_cm','product_width_cm']].prod(axis=1)
prod_aggs = order_items_prod.groupby('order_id').agg(
    peso_medio=('product_weight_g','mean'),
    volume_medio_cm3=('volume_cm3','mean'),
    categoria_mais_freq=('product_category_name', lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan)
).reset_index()

# Pagamentos: método principal e parcelas (agregar por pedido)
pay_aggs = payments.groupby('order_id').agg(
    payment_installments_max=('payment_installments','max'),
    payment_sequential_max=('payment_sequential','max'),
    payment_type_main=('payment_type', lambda s: s.mode().iloc[0] if not s.mode().empty else np.nan),
    payment_value_total=('payment_value','sum')
).reset_index()

# Clientes e vendedores: UF e cidade (one-hot posterior)
cust_cols = ['customer_id','customer_city','customer_state']
sel_cols = ['seller_id','seller_city','seller_state']

# Para obter UF do vendedor no nível do pedido, usar o vendedor do primeiro item
first_seller_per_order = order_items.sort_values('order_item_id').groupby('order_id').first().reset_index()[['order_id','seller_id']]
first_seller_per_order = first_seller_per_order.merge(sellers[sel_cols], on='seller_id', how='left')

# Montar base no nível do pedido
base = orders.merge(item_aggs, on='order_id', how='left') \
             .merge(prod_aggs, on='order_id', how='left') \
             .merge(pay_aggs, on='order_id', how='left') \
             .merge(customers[['customer_id','customer_state','customer_city']], on='customer_id', how='left') \
             .merge(first_seller_per_order[['order_id','seller_state','seller_city']], on='order_id', how='left')

# Features de tempo conhecidas cedo
base['dias_estimados'] = (base['order_estimated_delivery_date'] - base['order_purchase_timestamp']).dt.days
base['dias_ate_aprovacao'] = (base['order_approved_at'] - base['order_purchase_timestamp']).dt.total_seconds()/3600.0

# Evitar vazamento: não usar datas pós-entrega como features
feature_drop = ['order_delivered_carrier_date','order_delivered_customer_date']
base = base.drop(columns=[c for c in feature_drop if c in base.columns])

# Rácios úteis
base['ratio_frete_preco'] = base['total_frete'] / (base['total_preco'] + 1e-6)

# Alvo
base['y'] = base['atraso_entrega'].astype('int')

# Selecionar colunas para modelagem
numeric_cols = [
    'total_preco','total_frete','num_itens','num_vendedores','num_produtos',
    'peso_medio','volume_medio_cm3','payment_installments_max','payment_sequential_max',
    'payment_value_total','dias_estimados','dias_ate_aprovacao','ratio_frete_preco'
]
cat_cols = ['payment_type_main','categoria_mais_freq','customer_state','seller_state']

use_cols = numeric_cols + cat_cols + ['y']
df_model = base[use_cols].copy()

print(df_model.shape)
df_model.head()

TypeError: Must provide 'func' or tuples of '(column, aggfunc).

In [ ]:
# Separar treino e teste (estratificado)
from sklearn.utils import shuffle

# Remover linhas com alvo ausente
_df = df_model.dropna(subset=['y']).copy()

X = _df.drop(columns=['y'])
y = _df['y']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

numeric_features = [c for c in numeric_cols if c in X_train.columns]
categorical_features = [c for c in cat_cols if c in X_train.columns]

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

X_train.shape, X_test.shape, y_train.value_counts(normalize=True)

## 3) Análise Exploratória dos Dados (EDA)

Nesta seção exploramos:
- Distribuição do alvo (balanceamento de classes).
- Estatísticas descritivas (numéricas) e nulos.
- Histogramas/boxplots comparando atraso vs não atraso.
- Correlações entre variáveis numéricas.
- Frequência das principais categorias e taxa de atraso por categoria.

In [ ]:
# Distribuição do alvo
fig, ax = plt.subplots(1,2, figsize=(10,4))
y.value_counts().plot(kind='bar', ax=ax[0], title='Contagem por classe (0=NoPrazo, 1=Atraso)')
y.value_counts(normalize=True).plot(kind='bar', ax=ax[1], title='Proporção por classe')
plt.tight_layout()
plt.show()

# Nulos por coluna
nulls = df_model.isna().mean().sort_values(ascending=False)
print('Top 10 nulos:')
print(nulls.head(10))

# Estatísticas descritivas
num_desc = df_model[numeric_cols].describe().T
num_desc.head()

In [ ]:
# Correlações numéricas
plt.figure(figsize=(10,8))
num_df = df_model[numeric_cols].copy()
num_df = num_df.fillna(num_df.median(numeric_only=True))
sns.heatmap(num_df.corr(numeric_only=True), cmap='Blues', annot=False)
plt.title('Correlação entre variáveis numéricas')
plt.show()

# Taxa de atraso por categoria e UF (top categorias)
if 'categoria_mais_freq' in df_model.columns:
    cat_rate = df_model.groupby('categoria_mais_freq')['y'].mean().sort_values(ascending=False).head(15)
    cat_rate.plot(kind='bar', figsize=(10,4), title='Taxa de atraso por categoria (Top 15)')
    plt.show()

for col in ['payment_type_main','customer_state','seller_state']:
    if col in df_model.columns:
        rate = df_model.groupby(col)['y'].mean().sort_values(ascending=False).head(15)
        rate.plot(kind='bar', figsize=(10,4), title=f'Taxa de atraso por {col} (Top 15)')
        plt.show()

## 4) Treinamento do modelo

Vamos construir um pipeline com pré-processamento e dois modelos candidatos:
- Regressão Logística (class_weight='balanced')
- Random Forest (class_weight='balanced_subsample')

Usaremos GridSearchCV (StratifiedKFold=5, scoring='f1') para selecionar hiperparâmetros.

In [ ]:
# Definir pipelines e grades
pipe_lr = Pipeline(steps=[('preprocess', preprocessor),
                         ('clf', LogisticRegression(max_iter=1000, class_weight='balanced', random_state=RANDOM_STATE))])

pipe_rf = Pipeline(steps=[('preprocess', preprocessor),
                         ('clf', RandomForestClassifier(class_weight='balanced_subsample', random_state=RANDOM_STATE))])

param_grid_lr = {
    'clf__C': [0.1, 1.0, 3.0],
    'clf__penalty': ['l2'],
    'clf__solver': ['lbfgs']
}

param_grid_rf = {
    'clf__n_estimators': [150, 300],
    'clf__max_depth': [None, 10, 20],
    'clf__max_features': ['sqrt', 'log2']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

def run_grid(pipe, param_grid, name):
    grid = GridSearchCV(pipe, param_grid, cv=cv, scoring='f1', n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)
    print(f"{name} - melhor F1 CV: {grid.best_score_:.4f}")
    print('Melhores params:', grid.best_params_)
    return grid

res_lr = run_grid(pipe_lr, param_grid_lr, 'LogisticRegression')
res_rf = run_grid(pipe_rf, param_grid_rf, 'RandomForest')

best_grid = res_rf if res_rf.best_score_ >= res_lr.best_score_ else res_lr
best_name = 'RandomForest' if best_grid is res_rf else 'LogisticRegression'

print('Modelo escolhido:', best_name)

best_model = best_grid.best_estimator_

# Reajustar no treino completo (já é feito no GridSearch por padrão em refit=True)
# Salvar pipeline
joblib.dump(best_model, MODELS_DIR / 'best_pipeline.joblib')
(MODELS_DIR / 'best_pipeline.joblib').exists()

## 5) Avaliação do modelo e conclusão

Nesta seção:
- Avaliaremos o melhor modelo em teste (métricas e gráficos).
- Ajustaremos limiar de decisão (opcional) para maximizar F1.
- Registraremos conclusões objetivas e próximos passos.

In [ ]:
# Avaliação em teste
best_model = joblib.load(MODELS_DIR / 'best_pipeline.joblib')

# Predições padrão (threshold=0.5)
y_prob = best_model.predict_proba(X_test)[:,1]
y_pred = (y_prob >= 0.5).astype(int)

metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred, zero_division=0),
    'recall': recall_score(y_test, y_pred, zero_division=0),
    'f1': f1_score(y_test, y_pred, zero_division=0),
    'roc_auc': roc_auc_score(y_test, y_prob)
}
print(metrics)
print('\nClassification report:')
print(classification_report(y_test, y_pred, digits=4))

# Matriz de confusão
cm = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(1,2, figsize=(12,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax[0])
ax[0].set_title('Matriz de confusão (absoluta)')

cm_norm = confusion_matrix(y_test, y_pred, normalize='true')
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='Blues', ax=ax[1])
ax[1].set_title('Matriz de confusão (normalizada)')
plt.tight_layout()
plt.show()

# Curvas ROC e PR
disp = RocCurveDisplay.from_predictions(y_test, y_prob)
plt.show()

PrecisionRecallDisplay.from_predictions(y_test, y_prob)
plt.show()

# Opcional: busca de limiar por F1
thresholds = np.linspace(0.2, 0.8, 25)
best_thr, best_f1 = 0.5, metrics['f1']
for t in thresholds:
    _pred = (y_prob >= t).astype(int)
    _f1 = f1_score(y_test, _pred, zero_division=0)
    if _f1 > best_f1:
        best_f1, best_thr = _f1, t
print(f'Melhor threshold por F1: {best_thr:.3f} | F1={best_f1:.4f}')

# Importâncias / Coeficientes
clf = best_model.named_steps['clf']
if hasattr(clf, 'feature_importances_'):
    # Recuperar nomes das features após OneHot
    ohe = best_model.named_steps['preprocess'].named_transformers_['cat'].named_steps['onehot']
    cat_names = list(ohe.get_feature_names_out(categorical_features))
    feat_names = numeric_features + cat_names
    importances = pd.Series(clf.feature_importances_, index=feat_names).sort_values(ascending=False).head(20)
    importances.plot(kind='barh', figsize=(8,6), title='Importâncias das features (Top 20)')
    plt.gca().invert_yaxis()
    plt.show()
elif hasattr(clf, 'coef_'):
    ohe = best_model.named_steps['preprocess'].named_transformers_['cat'].named_steps['onehot']
    cat_names = list(ohe.get_feature_names_out(categorical_features))
    feat_names = numeric_features + cat_names
    coefs = pd.Series(clf.coef_[0], index=feat_names).sort_values(key=lambda s: s.abs(), ascending=False).head(20)
    coefs.plot(kind='barh', figsize=(8,6), title='Coeficientes (Top 20 por |coef|)')
    plt.gca().invert_yaxis()
    plt.show()

# Salvar métricas
pd.Series(metrics).to_csv(OUTPUT_DIR / 'test_metrics.csv')
print('Métricas salvas em output/test_metrics.csv')

### Conclusões e próximos passos (anotações)

- Classe minoritária: a taxa de atraso pode ser menor do que a de entregas no prazo; por isso priorizamos F1 e adotamos class_weight.
- Features relevantes típicas: dias_estimados, razão frete/preço, número de itens e características físicas têm impacto no atraso.
- Modelo escolhido: comparar F1 em validação cruzada; frequentemente RF supera LR pela não-linearidade.
- Limitações: potenciais vieses por UF/cidade; ausência de distância geográfica precisa; possibilidade de sazonalidade (deveria-se validar temporalmente).
- Próximos passos:
  - Engenhar distância cliente–vendedor (usando geolocalização) e variáveis temporais (mês, dia_da_semana, feriados).
  - Tuning adicional e validação temporal (time series split) para robustez.
  - Calibração de probabilidades (CalibratedClassifierCV) e otimização de threshold por objetivo de negócio.
  - Monitoramento de drift e re-treino periódico.